# Frequency analysis module

In [ ]:
# Basic imports
import xarray as xr
import hvplot.xarray
import numpy as np
import xdatasets as xd

import xhydro as xh
import xhydro.frequency_analysis as xhfa

## Extracting and preparing the data
For this example, we'll conduct a frequency analysis using historical time series from various sites. We begin by obtaining a dataset comprising hydrological information. Here, we use the [xdataset](https://hydrologie.github.io/xdatasets/notebooks/getting_started.html) library to acquire hydrological data from the [Ministère de l'Environnement, de la Lutte contre les changements climatiques, de la Faune et des Parcs](https://www.cehq.gouv.qc.ca/atlas-hydroclimatique/stations-hydrometriques/index.htm) in Quebec, Canada. Specifically, our query focuses on stations with IDs beginning with `020`, possessing a natural flow pattern and limited to streamflow data. 

Users may prefer to generate their own `xarray.DataArray` using their individual dataset. At a minimum, the `xarray.DataArray` used for frequency analysis has to follow these principles:

- The dataset needs a `time` dimension.
- If there is a spatial dimension, such as `id` in the example below, it needs an attribute `cf_role` with `timeseries_id` as its value.
- The variable will at the very least need a `units` attribute, although other attributes such as `long_name` and `cell_methods` are also expected by `xclim` (which is called at various points during the frequency analysis) and warnings will be generated if they are missing.

In [ ]:
ds = xd.Query(
    **{
        "datasets":{
            "deh":{
                "id" :["0510*"],
                "variables":["streamflow"],
            }
        }, "time":{"start": "1970-01-01", 
                   "minimum_duration":(15*365, 'd')},

  }
).data.squeeze().load()

# This dataset lacks some of the aforementioned attributes, so we need to add them.
ds["id"].attrs["cf_role"] = "timeseries_id"
ds["streamflow"].attrs = {"long_name": "Streamflow", "units": "m3 s-1", "standard_name": "water_volume_transport_in_river_channel", "cell_methods": "time: mean"}

ds

In [ ]:
ds.streamflow.dropna('time', how='all').hvplot(x='time', grid=True, widget_location='bottom', groupby='id')

## Customizing the analysis settings

### a) Defining seasons
We can define seasons using indexers that are compatible with `xclim.core.calendar.select_time`. There are currently four accepted types of indexers:

- `month`, followed by a sequence of month numbers.
- `season`, followed by one or more of 'DJF', 'MAM', 'JJA' and 'SON'.
- `doy_bounds`, followed by a sequence representing the inclusive bounds of the period to be considered (start, end).
- `date_bounds`, which is the same as above, but using a month-day (%m-%d) format.

For the purpose of getting block maxima through `xhydro.indicators.get_yearly_op`, the indexers need to be grouped within a dictionary, with the key being the label to be given to the requested period of the year. A second key can be used to instruct on the resampling frequency, for example to wrap around the year for winter.

In [ ]:
# Some examples
timeargs = {
    "spring": {"date_bounds": ["02-11", "06-19"]},
    "summer": {"doy_bounds": [152, 243]},
    "fall": {"month": [9, 10, 11]},
    "winter": {"season": ['DJF'], "freq": "YS-DEC"},  # To correctly wrap around the year, we need to specify the resampling frequency.
    "august": {"month": [8]},
    "annual": {}
    }

### b) Getting block maxima
Upon selecting each desired season, we can extract block maxima timeseries from every station using `xhydro.indicators.get_yearly_op`. The main arguments are:

- `op`: the operation to compute. One of "max", "min", "mean", "sum".
- `input_var`: the name of the variable. Defaults to "streamflow".
- `window`: the size of the rolling window. A "mean" is performed on the rolling window prior to the `op` operation.
- `timeargs`: as defined previously. Leave at `None` to get the annual maxima.
- `missing` and `missing_options`: to define tolerances for missing data. See [this page](https://xclim.readthedocs.io/en/stable/checks.html#missing-values-identification) for more information.
- `interpolate_na`: whether to interpolate missing data prior to the `op` operation. Only used for `sum`.

The function returns a `xarray.Dataset` with 1 variable per indexer.

In [ ]:
# Here, we hide years with more than 15% of missing data.
ds_4fa = xh.indicators.get_yearly_op(ds, op="max", timeargs=timeargs, missing="pct", missing_options={"tolerance": 0.15})

ds_4fa

In [ ]:
ds_4fa.streamflow_max_summer.dropna('time', how='all').hvplot(x='time', grid=True, widget_location='bottom', groupby='id')

### c) Using custom seasons per year or per station

Using individualized date ranges for each year or each catchment is not explicitely supported, so users should instead mask their data prior to calling `get_yearly_op`. Note that when doing this, `missing` should be adjusted accordingly.

In [ ]:
# Create a mask beforehand
import random
nyears = np.unique(ds.time.dt.year).size
dom_start = xr.DataArray(np.random.randint(1, 30, size=(nyears, )).astype("str"), dims=("year"), coords={"year": np.unique(ds.time.dt.year)})
dom_end = xr.DataArray(np.random.randint(1, 30, size=(nyears, )).astype("str"), dims=("year"), coords={"year": np.unique(ds.time.dt.year)})

mask = xr.zeros_like(ds["streamflow"])
for y in dom_start.year.values:
    # Random mask of dates per year, between April and June.
    mask.loc[{"time": slice(str(y)+"-04-"+str(dom_start.sel(year=y).item()), str(y)+"-06-"+str(dom_end.sel(year=y).item()))}] = 1

In [ ]:
mask.hvplot(x='time', grid=True, widget_location='bottom', groupby='id')

In [ ]:
# The name of the indexer will be used to identify the variable created here
timeargs_custom = {"custom": {}}

# We use where() to mask the data that we want to ignore
masked = ds.where(mask==1)
# Since we masked almost all of the year, our tolerance for missing data should be changed accordingly
missing = "at_least_n"
missing_options = {"n": 45}

# We use xr.merge() to combine the results with the previous dataset.
ds_4fa = xr.merge([ds_4fa, xh.indicators.get_yearly_op(masked, op="max", timeargs=timeargs_custom, missing=missing, missing_options=missing_options)])

In [ ]:
ds_4fa.streamflow_max_custom.dropna('time', how='all').hvplot(x='time', grid=True, widget_location='bottom', groupby='id')

### d) Computing volumes

The frequency analysis can also be performed on volumes, using a similar workflow. The main difference is that if we're starting from streamflow, we'll first need to convert them into volumes using `xhydro.indicators.compute_volume`. Also, if required, `get_yearly_op` has an argument `interpolate_na` that can be used to interpolate missing data prior to the sum.

In [ ]:
# Get a daily volume from a daily streamflow
ds["volume"] = xh.indicators.compute_volume(ds["streamflow"], out_units="hm3")

# We'll take slightly different indexers
timeargs_vol = {
    "spring": {"date_bounds": ["04-30", "06-15"]},
    "annual": {}
    }

# The operation that we want here is the sum, not the max.
ds_4fa = xr.merge([ds_4fa, xh.indicators.get_yearly_op(ds, op="sum", input_var="volume", timeargs=timeargs_vol, missing="pct", missing_options={"tolerance": 0.15}, interpolate_na=True)])

In [ ]:
ds_4fa.volume_sum_spring.dropna('time', how='all').hvplot(x='time', grid=True, widget_location='bottom', groupby='id')

## Local frequency analysis

Once we have our yearly maximums (or volumes/minimums), the first step in a local frequency analysis is to call `xhfa.local.fit` to obtain distribution parameters. The options are:

- `distributions`: list of [SciPy distributions](https://docs.scipy.org/doc/scipy/reference/stats.html#continuous-distributions). Defaults to ["expon", "gamma", "genextreme", "genpareto", "gumbel_r", "pearson3", "weibull_min"].
- `min_years`: minimum number of years required to fit the data.
- `method`: fitting method. Defaults to the maximum likelihood.

In [ ]:
# To speed up the Notebook, we'll only perform the analysis on a subset of variables
params = xhfa.local.fit(ds_4fa[["streamflow_max_spring", "volume_sum_spring"]], min_years=15)

params

Information Criteria such as the AIC, BIC, and AICC are useful to determine which statistical distribution is better suited to a given location. These three criteria can be computed using `xhfa.local.criteria`.

In [ ]:
criteria = xhfa.local.criteria(ds_4fa[["streamflow_max_spring", "volume_sum_spring"]], params)

criteria

Finally, return periods can be obtained using `xhfa.local.parametric_quantiles`. The options are:

- `t`: the return period(s) in years.
- `mode`: whether the return period is the probability of exceedance (max) or non-exceedance (min). Defaults to `max`.

In [ ]:
rp = xhfa.local.parametric_quantiles(params, t=[20, 100])

rp.load()

In a future release, plotting will be handled by a proper function. For now, we'll show an example in this Notebook using preliminary utilities.

`xhfa.local._prepare_plots` generates datapoints required to plot the results of the frequency analysis. If `log=True`, it will return log-spaced x values between `xmin` and `xmax`.


In [ ]:
data = xhfa.local._prepare_plots(params, xmin=1, xmax = 1000, npoints=50, log=True)
data.load()

`xhfa.local._get_plotting_positions` allows you to get plotting positions for all variables in the dataset.  It accepts `alpha` `beta` arguments. See the [SciPy documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mstats.plotting_positions.html) for typical values. By default, (0.4, 0.4) will be used, which corresponds to approximately quantile unbiased (Cunnane).

    

In [ ]:
pp = xhfa.local._get_plotting_positions(ds_4fa[['streamflow_max_spring']])
pp

In [ ]:
#Lets plot the observations
p1 = data.streamflow_max_spring.hvplot(x='return_period', by='scipy_dist', grid=True, groupby=['id'], logx=True) 

In [ ]:
#Lets now plot the distributions
p2 = pp.hvplot.scatter(x='streamflow_max_spring_pp',y='streamflow_max_spring', grid=True, groupby=['id'], logx=True)

In [ ]:
#And now combining the plots
p1 * p2